# TODO GO to KRM and create a volume first with name "volume-sentinel"

In krm create first a volume otherwise the RUN is stuck waiting for the volume
persisten_volume_claim specs:
- volume_name : volume-sentinel # this name is to be equal to the "name parameter inside volumes during the run of the function"
- space : 15Gbi
- storage_class_name: #disk
- access_mode: readWriteOnce

If you have any doubts on the params check the README.md inside https://github.com/tn-aixpa/sentinel-tools 

In [1]:
import digitalhub as dh
# conda install -c conda-forge gdal
PROJECT_NAME = "docker-sentinel"
proj = dh.get_or_create_project(PROJECT_NAME) # source="git://github.com/scc-digitalhu

In [ ]:
# THIS NEED TO BE EXECUTED JUST ONCE
secret0 = proj.new_secret(name="CDSETOOL_ESA_USER", secret_value="esa_username")
secret1 = proj.new_secret(name="CDSETOOL_ESA_PASSWORD", secret_value="esa_password")

In [ ]:
string_dict_data = """{
  'satelliteParams':{
      'satelliteType': 'Sentinel1',
      'processingLevel': 'LEVEL1',
      'sensorMode': 'IW',
      'productType': 'SLC'
  } ,
  'startDate': '2023-12-12',
  'endDate': '2023-12-13',
  'geometry': 'POLYGON((10.98014831542969 45.455314263477874,11.030273437500002 45.44808893044964,10.99937438964844 45.42014226680115,10.953025817871096 45.435803739956725,10.98014831542969 45.455314263477874))',
  'area_sampling': 'True',
  'tmp_path_same_folder_dwl':'True',
  'artifact_name': 'directory_name_inside_s3',
  's3_path': 's3://{my-bucket}/{path_to_artifact}/'
  }"""

# s3 path is not mandatory

list_args =  ["main.py",string_dict_data]
function = proj.new_function("donwload",kind="container",image="alattaruolo/sentinel-basic:v0.0.23",command="python",args=list_args)

In [ ]:
run = function.run(action="job",
secrets=["CDSETOOL_ESA_USER","CDSETOOL_ESA_PASSWORD"],
#envs=[{"name":"CDSETOOL_ESA_USER","value":"esa_user"},{"name":"CDSETOOL_ESA_PASSWORD","value":"esa_password"}],
volumes=[{
    "volume_type": "persistent_volume_claim",
    
    "name": "volume-sentinel", # this name has to be equal to the name of the volume created in krm
    "mount_path": "/files",
    "spec": {
        "claim_name": "volume-sentinel"
    }}],)

In [9]:
# import os
# def create_json_from_env():
#     result = {}
#     result['PROJECT_NAME'] = os.environ["PROJECT_NAME"]
#     result['S3_BUCKET_NAME'] = os.environ["S3_BUCKET_NAME"]
#     result['S3_ENDPOINT_URL'] = os.environ["S3_ENDPOINT_URL"]
#     result['DIGITALHUB_CORE_ENDPOINT'] = os.environ["DIGITALHUB_CORE_ENDPOINT"]
#     result['AWS_ACCESS_KEY_ID'] = os.environ["AWS_ACCESS_KEY_ID"]
#     result['AWS_SECRET_ACCESS_KEY'] = os.environ["AWS_SECRET_ACCESS_KEY"]
#     return result

In [ ]:
# create_json_from_env()